In [44]:
import opensimplex
import numpy as np
import matplotlib.pyplot as plt
import math
from random import randint
import os
import multiprocessing as mp
import plotly.express as px
from random import randint

biomas = np.empty(shape=(128,128,128), dtype=np.uint8)
for i in range(128):
    for j in range(128):
        for k in range(128):
            mejor = float('inf')
            for r in generator.BIOMAS.keys():
                a = np.linalg.norm(np.array(generator.BIOMAS[r][0]) - np.array([i/128, j/128, k/128]))
                if a < mejor:
                    mejor = a
                    biomas[i, j, k] = r
    print(i)
print(biomas)
np.save(f"CERCANIA_BIOMAS.npy", biomas)


In [25]:
class generator:
    #(Humedad altura temperatura), nombre del bioma, color, {objetos que pueden aparecer, probabilidades}
    BIOMAS = {
        0: ([-1, -1, -1], 'Oceano', (56, 148, 194), {}),
        1: ([-1, -1, -1], 'Oceano profundo', (27, 59, 140), {}),
        2: ([-1, -1, -1], 'Rios y lagos', (126, 180, 237), {}),
        3: ([0.5, 0.0, 0.5], 'Costa', (245, 240, 108), {"Palmera": 0.7, "Piedra pequeña": 0.3}),
        4: ([0.85, 0.05, 0.45], 'Marismas', (171, 222, 32), {"Arbusto": 1}),
        5: ([0.75, 0.1, 0.05], 'Polo', (250, 250, 250), {"Tempano": 0.5, "Piedra congelada": 0.5}),
        6: ([0.8, 0.15, 0.45], 'Pantano', (33, 133, 99), {"Arbusto": 0.2, "Piedra pequeña": 0.2, "Arbol": 0.6}),
        7: ([0.5, 0.2, 0.5], 'Bosque', (53, 118, 43), {"Arbol": 0.6, "Arbusto": 0.2, "Flores": 0.2}),
        8: ([0.6, 0.2, 0.55], 'Selva', (0, 183, 10), {"Arbol": 0.7, "Helecho": 0.3}),
        9: ([0.35, 0.2, 0.55], 'Dehesa', (128, 179, 30), {"Arbol": 0.5, "Arbusto": 0.5}),
        10: ([0.2, 0.2, 0.6], 'Savana', (228, 125, 28), {"Arbol": 0.3, "Arbusto": 0.7}),
        12: ([0.75, 0.25, 0.75], 'Termales', (39, 156, 121), {"Geiser": 0.5, "Piedra pequeña": 0.5}),
        11: ([0.0, 0.3, 0.8], 'Desierto', (255, 211, 29), {"Cactus": 0.7, "Arbusto muerto": 0.3}),
        13: ([0.4, 0.4, 0.4], 'Valle', (144, 219, 148), {"Arbol": 0.8, "Arbusto": 0.2}),
        14: ([0.2, 0.65, 0.55], 'Meseta', (110, 88, 66), {"Roca grande": 0.1, "Arbusto": 0.5, "Roca pequeña": 0.4}),
        15: ([0.7, 0.8, 0.2], 'Tundra', (208, 208, 240), {"Piedra pequeña": 0.5, "Flores": 0.5}),
        16: ([0.15, 0.8, 0.85], 'Volcanico', (159, 16, 16), {"Arbol carbonizado": 0.6, "Charco de lava": 0.4}),
        17: ([0.35, 9, 0.35], 'Montaña', (125, 125, 125), {"Roca grande": 0.3, "Roca pequeña": 0.7}),
    }
    CHUNK_SIZE = 128
    CERCANIA_BIOMAS = np.load("./CERCANIA_BIOMAS.npy" )

    def __init__(self, seedTemp = None, seedAltu = None, seedHume = None):
        self.SEEDTEMP = seedTemp if seedTemp != None else randint(-2**30, 2**30)
        self.SEEDALTU = seedAltu if seedTemp != None else randint(-2**30, 2**30)
        self.SEEDHUME = seedHume if seedTemp != None else randint(-2**30, 2**30)

    def getNoise(self, seed: int, x_in: int, y_in: int, iterations: int, size: int, disparity: int = 3):
        opensimplex.seed(seed)
        ruido = 0
        for r in range(iterations):
            divisor = (size/2**r)
            exponenciador = (2**r)
            ruido += opensimplex.noise2(x=(x_in)/divisor, y=(y_in)/divisor)/exponenciador
            
        maximo = sum([2/(2**r) for r in range(iterations)])
        ruido = (ruido + maximo)/(2*maximo)     
        for r in range(disparity):
            ruido = 0.5 + math.sin(math.pi*ruido - math.pi/2)/2
        return ruido

    def getNoiseArray(self, seed: int, x_in: int, y_in: int, iterations: int, size: int, disparity: int = 3):
        """
        Generates a 2D noise array using OpenSimplex noise with multi-level detail and smoothing.

        Args:
            seed (int): The seed used to initialize the noise generation.
            x_in (int): The x-coordinate index for the chunk, which determines the chunk's location in the noise grid.
            y_in (int): The y-coordinate index for the chunk, which determines the chunk's location in the noise grid.
            iterations (int): The number of levels of detail or layers to add to the noise. Higher values create more detailed noise.
            size (int): The scale factor for the noise. Larger values make the noise features larger.
            disparity (int, optional): The number of smoothing iterations applied to the generated noise values. Defaults to 3.

        Returns:
            np.ndarray: A 2D array of floats representing the generated noise values, normalized and smoothed.

        Details:
            1. The noise is generated using the OpenSimplex algorithm. For each iteration, the noise is calculated at different scales 
            and added to the `noise_array` with decreasing impact for each iteration.
            2. The noise values are normalized based on the maximum possible summed noise value across all iterations.
            3. After generating the noise, the function applies sinusoidal smoothing multiple times (based on the `disparity` value) to 
            create smoother transitions in the noise.
            4. The output is a chunk of noise, determined by the size of the chunk (`self.CHUNK_SIZE`) and its position (`x_in`, `y_in`).

        Example:
            noise = getNoiseArray(seed=42, x_in=1, y_in=2, iterations=4, size=100, disparity=3)
        """
        x = self.CHUNK_SIZE*x_in
        y = self.CHUNK_SIZE*y_in
        opensimplex.seed(seed)

        rango = range(0,self.CHUNK_SIZE)
        noise_array = np.zeros(dtype=float, shape=(self.CHUNK_SIZE, self.CHUNK_SIZE))

        for r in range(iterations):
            divisor = (size/2**r)
            exponenciador = (2**r)
            for i in rango:
                for j in rango:
                    noise_array[j, i] += opensimplex.noise2(x=(i+x)/divisor, y=(y+j)/divisor)/exponenciador
        
        maximo = sum([2/(2**r) for r in range(iterations)])
        for i in rango:
            for j in rango:
                valor = (noise_array[j, i] + maximo)/(2*maximo)     
                for r in range(disparity):
                    valor = 0.5 + math.sin(math.pi*valor - math.pi/2)/2
                noise_array[j, i] = valor
        return noise_array
    
    def getBioma(self, temp, altu, hume):
        """
        Determines the biome type based on temperature, altitude, and humidity.

        Args:
            temp (float): The temperature value, expected to be in the range [0, 1].
            altu (float): The altitude value, expected to be in the range [0, 1].
            hume (float): The humidity value, expected to be in the range [0, 1].

        Returns:
            int: An integer representing the biome type.
                 0 - Ocean
                 1 - Deep Ocean
                 Other values are determined by the CERCANIA_BIOMAS lookup table.
        """
        if(altu < 0.5):
            if (altu <= 0.4):
                return 0 #Oceano
            else:
                return 1 #Oceano profundo
        else:
            alt = math.floor(128*(altu-0.5)*2)
            temp = math.floor(128*temp)
            hum = math.floor(128*hume)
            return self.CERCANIA_BIOMAS[hum, alt, temp]

    def getChunk(self, x: int, y: int):
        """
        Retrieves or generates a chunk of terrain data based on the given coordinates.

        If the chunk data already exists in the specified file path, it loads and returns the data.
        Otherwise, it generates new terrain data using Perlin noise, saves it to the file path, and returns the generated data.

        Parameters:
        x (int): The x-coordinate of the chunk.
        y (int): The y-coordinate of the chunk.

        Returns:
        numpy.ndarray: A 2D array representing the terrain data for the specified chunk.
        """
        if os.path.exists(f"./Chunks/T_{self.SEEDTEMP}A_{self.SEEDTEMP}H_{self.SEEDTEMP}/{x}/{y}.npy"):
            return np.load(f"./Chunks/T_{self.SEEDTEMP}A_{self.SEEDTEMP}H_{self.SEEDTEMP}/{x}/{y}.npy")
        else:
            temp = self.getNoiseArray(self.SEEDTEMP, x, y, 3, 128, 2)
            altu = self.getNoiseArray(self.SEEDALTU, x, y, 6, 512, 4)
            hume = self.getNoiseArray(self.SEEDHUME, x, y, 3, 256, 2)
            array_biomas = np.zeros(dtype=np.uint8, shape=(self.CHUNK_SIZE,self.CHUNK_SIZE))

            for i in range(self.CHUNK_SIZE):
                for j in range(self.CHUNK_SIZE):
                    array_biomas[i, j] = self.getBioma(hume[i, j], altu[i, j], temp[i, j])
            os.makedirs(f"./Chunks/T_{self.SEEDTEMP}A_{self.SEEDTEMP}H_{self.SEEDTEMP}/{x}", exist_ok=True)
            np.save(f"./Chunks/T_{self.SEEDTEMP}A_{self.SEEDTEMP}H_{self.SEEDTEMP}/{x}/{y}", array_biomas)
            return array_biomas
    
    def getChunksInRange(self, x_range: tuple[int, int], y_range: tuple[int, int]):
        """
        Generates a 2D array of chunks within the specified x and y ranges.
        This method creates and starts multiple processes to generate chunks in parallel,
        then joins the processes and combines the resulting chunks into a single 2D array.
        Args:
            x_range (tuple[int, int]): A tuple specifying the start and end of the x range.
            y_range (tuple[int, int]): A tuple specifying the start and end of the y range.
        Returns:
            numpy.ndarray: A 2D array containing the combined chunks.
        """
        arr = None
        for i in range(x_range[0], x_range[1]):
            arr_proc = []
            for j in range(y_range[0], y_range[1]):
                arr_proc.append(mp.Process(target=self.getChunk, args=(i, j)))
                arr_proc[-1].start()
            for proc in arr_proc:
                proc.join()

            arr_line = None
            for j in range(y_range[0], y_range[1]):
                if arr_line is None:
                    arr_line = self.getChunk(i, j)
                else:
                    arr_line = np.vstack((arr_line, self.getChunk(i, j)))

            if arr is None:
                arr = arr_line
            else:
                arr = np.hstack((arr, arr_line))
        return arr

    def representation(self, x_range: tuple[int, int], y_range: tuple[int, int]):
        """
        Generates a visual representation of the terrain based on Perlin noise.
        Args:
            x_range (tuple[int, int]): The range of x-coordinates to generate the terrain for.
            y_range (tuple[int, int]): The range of y-coordinates to generate the terrain for.
        Returns:
            np.ndarray: A 3D numpy array representing the RGB values of the terrain.
        The function performs the following steps:
        1. Retrieves the terrain chunks within the specified x and y ranges.
        2. Initializes an empty array to store the RGB values of the terrain.
        3. Iterates through the terrain chunks and assigns the corresponding RGB values from the BIOMAS dictionary.
        4. Creates an image of the terrain using Plotly's imshow function.
        5. Adjusts the figure size to match the dimensions of the terrain array.
        6. Adds a legend to the figure, representing different biomes with their respective colors.
        7. Displays the figure.
        8. Returns the RGB array of the terrain.
        """
        arr = self.getChunksInRange(x_range, y_range)
        arr2 = np.zeros((arr.shape[0], arr.shape[1], 3), dtype=np.uint8)

        for i in range(arr.shape[0]):
            for j in range(arr.shape[1]):
                arr2[i, j] = generator.BIOMAS[arr[i, j]][2]
        
        fig = px.imshow(arr2)
        fig.update_layout(width=arr.shape[0], height=arr.shape[1])  # Adjust the figure size here

        # Add legend
        legend_items = []
        for key, value in generator.BIOMAS.items():
            legend_items.append(
                dict(
                    name=value[1],
                    marker=dict(color=f"rgb{value[2]}", size=20),
                    mode='markers',
                    type='scatter',
                    x=[None],
                    y=[None]
                )
            )
        fig.add_traces(legend_items)
        fig.show()
        return arr2

In [20]:
a = generator(1,2,3)

In [21]:
a.representation((0,4),(0,4))

array([[[245, 240, 108],
        [245, 240, 108],
        [245, 240, 108],
        ...,
        [ 56, 148, 194],
        [ 56, 148, 194],
        [ 56, 148, 194]],

       [[ 27,  59, 140],
        [ 27,  59, 140],
        [ 27,  59, 140],
        ...,
        [ 56, 148, 194],
        [ 56, 148, 194],
        [ 56, 148, 194]],

       [[ 27,  59, 140],
        [ 27,  59, 140],
        [ 27,  59, 140],
        ...,
        [ 56, 148, 194],
        [ 56, 148, 194],
        [ 56, 148, 194]],

       ...,

       [[ 56, 148, 194],
        [ 56, 148, 194],
        [ 56, 148, 194],
        ...,
        [110,  88,  66],
        [110,  88,  66],
        [110,  88,  66]],

       [[ 56, 148, 194],
        [ 56, 148, 194],
        [ 56, 148, 194],
        ...,
        [110,  88,  66],
        [110,  88,  66],
        [110,  88,  66]],

       [[ 56, 148, 194],
        [ 56, 148, 194],
        [ 56, 148, 194],
        ...,
        [110,  88,  66],
        [110,  88,  66],
        [110,  88,  66]]

In [26]:
a = generator(1,2,3)
b = a.getChunk(0,0)
b_noise_Temp = a.getNoiseArray(1, 0, 0, 5, 128, 2)
b_noise_Altu = a.getNoiseArray(2, 0, 0, 3, 512, 4)
b_noise_Hume = a.getNoiseArray(3, 0, 0, 3, 256, 2)

In [29]:
print(b_noise_Altu[0,17])
print(a.getNoise(2,17,0,3,512,4))

0.5579839951516736
0.5579839951516736


In [51]:
def generate_random_matrix(object_number: int):
    array = np.zeros(shape=(4, 4))
    elements = set([r for r in range(16)])
    while len(elements) != object_number:
        actual = randint(0, 15)
        if actual in elements:
            elements.remove(actual)
    for i in elements:
        array[i // 4, i % 4] = 1
    return array

# Call the function and print the result
ggg = generate_random_matrix(9)
print(ggg)

[[0. 1. 1. 0.]
 [1. 1. 0. 1.]
 [1. 0. 0. 1.]
 [0. 1. 1. 0.]]


In [60]:
from collections import Counter

arrdemo = np.empty(shape=(b.shape[0], b.shape[1]), dtype=np.uint8)
for i in range(0, b.shape[0], 4):
    for j in range(0, b.shape[1], 4):
        ggg = generate_random_matrix(5)
        arrdemo[i:i+4, j:j+4] = ggg
print(arrdemo[0:20,0:20])

[[0 1 0 1 0 1 1 0 1 0 0 0 1 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 0]
 [0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 1 0]
 [1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0]
 [0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 1]
 [1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 0]
 [0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1]
 [1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1]
 [0 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0]
 [0 1 0 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0]
 [1 0 0 1 1 0 0 0 1 0 1 1 0 0 1 0 1 1 0 1]
 [0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0]
 [1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0]]
